# S2DS - Cleaning and feature extraction

This notebook guides through the process of
- __Querying__ the data from the Gophr database
- __Cleaning__ the data
- __Merging__ the different dataframes
- __Feature__ extraction from the cleaned data that can be used for prediction

After carrying out these steps, you're ready to switch over to the modelling part in the modelling notebooks.

The notebooks usese the expression 'dataframe' which refers to an object type from the pandas package in Python basically comparable to an SQL table.

## Preparation

Notebook settings that enable changes in the source code reflected here on the fly

In [1]:
# configure notebook settings
%load_ext autoreload
%autoreload 2

Load required Python packages from the conda environment 's2ds' - for cleaning and feature extraction, we only need pandas

In [2]:
# import Python packages
import pandas as pd
# Pandas by default shortens the output of larger tables, 
# use these options to get the full picture adjusted according to your needs
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_info_columns', 200)

Access packages from our s2ds source tree

In [3]:
# path and io utility helper functions
import utils
# data querying, cleaning and merging functions
import data  
# feature generating functions
import features

## Querying the database

All queries are saved as .feather files (see https://wesmckinney.com/blog/feather-arrow-future/)

Query the jobs and the jobs_extra table, aggregating the jobs_history table
and save a feather file where rows are jobs

In [4]:
# get jobs data
df_jobs = data.execute_query_and_save_df('df1_query.sql','jobs.feather')

Opening database connection
Querying database with query in c:\users\thilo\repos\s2ds\src\data\df1_query.sql
Closing database connection
Writing feather file to c:\users\thilo\repos\s2ds\data\raw\jobs.feather


Query the jobs_history table and save a feather file where rows are job history events

In [5]:
# get jobs history data
df_events = data.execute_query_and_save_df('query_jobs_history.sql', 'jobs_history.feather')

Opening database connection
Querying database with query in c:\users\thilo\repos\s2ds\src\data\query_jobs_history.sql
Closing database connection
Writing feather file to c:\users\thilo\repos\s2ds\data\raw\jobs_history.feather


Query the weather table and save a feather file with hourly weather information

In [6]:
# get weather data
df_weather = data.execute_query_and_save_df('weather_query.sql','weather.feather')

Opening database connection
Querying database with query in c:\users\thilo\repos\s2ds\src\data\weather_query.sql
Closing database connection
Writing feather file to c:\users\thilo\repos\s2ds\data\raw\weather.feather


## Reading the raw data

If you have queried the database already, you can skip the previous section "Querying the database" and continue here.
If you have just executed the previous section, you can skip this section.

In [7]:
# read jobs dataframe
df_jobs = pd.read_feather(utils.path_to('data', 'raw', 'jobs.feather'))

In [8]:
# read weather dataframe
df_weather = pd.read_feather(utils.path_to('data', 'raw', 'weather.feather'))

In [9]:
# read event dataframe
df_event = pd.read_feather(utils.path_to('data', 'raw', 'jobs_history.feather'))

## Cleaning the data

Call the jobs cleaning function.

For a closer look at the cleaning procedure, open the module __src/data/clean_dataframe_p1__.

#### Features
In the first part, in the private function ___clean_postcode__ the pickup and delivery postcodes are cleaned with regular expressions to have a valid London outcode as a result. In the postcode cleaning procedure, first the district is identified. Jobs without an identifiable districts are removed, also jobs with districts outside of London. Jobs with assigned London outcodes that can't be matched to existing outcodes also are removed. Everything is printed out during the postcode cleaning procedure.

In the second part, further cleaning takes place in the function __clean_dataframe_p1__. The col_dict in the cleaning function provides the list of variables for different cleaning procedures:
- __col_to_remove__ entries are dropped from the dataframe
- __col_fillna_int__ entries have missing values replaced by the value 99
- __col_keep_na__ entries are ignored when deleting all rows with any missing values
- __col_to_int__ are recoded as integers

Special treatment for 
- is_first_war_job: missing values are replaced by zero.
- war_job_id: All jobs with a non missing values (i.e. return jobs) are deleted.

All jobs with missing values are removed, except missing values in the columns in col_keep_na which are ignored. The column name with a count for the missing values is printed out one after another for that it's transparent why jobs are removed.

#### Outcome
Jobs with a status of 80 are counted as rejected, also if the query result counts no rejected events in abs_rejected.
Jobs with a canceled_reason other than 'NOT_ACCEPTED' are removed.

The 'hard' jobs threshold for model 2 is set hard coded to a rejected_count larger than 5.

In [10]:
# call cleaning function
df_jobs_cleaned = data.clean_dataframe_p1(df_jobs)


Recoding postcodes in pickup_postcode
Cleaning postcodes in pickup_postcode
Deleting 3 districts not recognized
LONDON     1
ECIV8BR    1
CRO4LP     1
Name: pickup_postcode, dtype: int64
Deleting 12990 districts outside London, listing districts with freq >= 100
RG    2076
HA    1878
KT    1636
GU    1248
EN    1083
SM    1034
RM     738
TW     688
CR     569
UB     284
M      192
TN     176
RH     166
SS     154
DA     152
WD     123
BR     110
IG     104
Name: district, dtype: int64
Deleting 1 unrecognized London outcodes
E215QR    1
Name: pickup_postcode, dtype: int64

Recoding postcodes in delivery_postcode
Cleaning postcodes in delivery_postcode
Deleting 1 districts not recognized
1G118RG    1
Name: delivery_postcode, dtype: int64
Deleting 6883 districts outside London, listing districts with freq >= 100
SM    1039
CR     995
TW     887
KT     548
EN     441
HA     358
UB     272
WD     239
BR     218
RM     186
SL     183
IG     170
DA     122
CM     104
GU     102
Name: distric

### Call weather cleaning function
Weather cleaning in data/clean_weahter_df removes some irrelevant columns and duplicate date entries from the weather dataframe

In [11]:
# call cleaning function
df_weather_cleaned = data.clean_weather_df(df_weather)

clean_weather_df(): drop 6 duplicates.


## Merging datasets

### Job level
Merge the dataframe for analyses on the job level and save it.

In [12]:
# merge jobs and weather
df_jobs_merged = data.merge_df1_and_weather(df_jobs_cleaned, df_weather_cleaned)
# save the dataframe
data.write_feather_file(df_jobs_merged, utils.path_to('data', 'final', 'df_clean_jobs.feather'))

Writing feather file to c:\users\thilo\repos\s2ds\data\final\df_clean_jobs.feather


,job_id,status,pickup_postcode,pickup_location_lat,pickup_location_lng,delivery_postcode,delivery_location_lat,delivery_location_lng,distance,insertion_date,earliest_pickup_time,delivery_deadline,date_booked,date_started,vehicle_type,courier_money_earned_net,is_first_war_job,job_priority,riskiness,show_on_board,size_x,size_y,size_z,weight,special_care,is_food,is_fragile,is_liquid,is_not_rotatable,is_glass,is_baked,is_flower,is_alcohol,is_beef,is_pork,canceled_status,canceled_reason,estimated_journey_time,courier_earnings_calc,event_counts,accepted_count,rejected_count,pickup_district,pickup_postcode_outer,pickup_postcode_inner,delivery_district,delivery_postcode_outer,delivery_postcode_inner,is_accepted,is_rejected,is_hard,dt_iso,temp,feels_like,humidity,wind_speed,clouds_all,weather_main,weather_icon,is_daytime
0,231558,99,SW3 4NX,51.489251,-0.164032,E20 1EJ,51.543535,-0.006004,16.328,2018-01-01 11:22:47,2018-01-01 13:20:00,2018-01-01 16:00:00,2018-01-01 13:16:12,2018-01-01 13:16:16,20,29.42,0,1,0,0,35.0,25.0,2.5,0.10,1,0,1,0,0,0,0,0,0,0,0,NaN,None,73.0,19.77,1,1,0,SW,SW3,4NX,E,E20,1EJ,1,0,0,2018-01-01 11:00:00,6.77,4.16,76,1.54,100,Rain,10d,1
1,231560,99,NW1 7JX,51.537842,-0.142225,N7 7BY,51.553447,-0.108974,4.196,2018-01-01 12:01:37,2018-01-01 12:15:00,2018-01-01 13:00:00,2018-01-01 12:02:38,2018-01-01 12:02:46,20,9.08,0,0,0,0,40.0,30.0,20.0,9.00,1,99,99,99,99,1,99,99,1,1,1,NaN,None,43.0,4.82,1,1,0,NW,NW1,7JX,N,N7,7BY,1,0,0,2018-01-01 12:00:00,7.19,5.38,76,0.51,90,Clouds,04d,1
2,231561,99,NW1 7JX,51.537842,-0.142225,W2 5AR,51.515672,-0.196826,5.834,2018-01-01 12:03:34,2018-01-01 15:15:00,2018-01-01 16:00:00,2018-01-01 12:03:35,2018-01-01 14:45:07,15,10.60,0,2,0,0,40.0,30.0,20.0,17.60,1,0,0,99,99,1,99,99,0,1,1,NaN,None,65.0,15.47,1,1,0,NW,NW1,7JX,W,W2,5AR,1,0,0,2018-01-01 12:00:00,7.19,5.38,76,0.51,90,Clouds,04d,1
3,231562,99,W1T 1BZ,51.518604,-0.132486,W12 7GF,51.508141,-0.219373,8.024,2018-01-01 13:04:13,2018-01-01 13:05:00,2018-01-01 14:45:00,2018-01-01 13:07:01,2018-01-01 13:07:07,20,18.21,0,1,0,0,25.0,16.5,0.5,0.75,0,0,0,0,0,0,0,0,0,0,0,NaN,None,53.0,13.13,3,1,2,W,W1T,1BZ,W,W12,7GF,1,0,0,2018-01-01 13:00:00,7.12,2.58,81,4.63,100,Rain,10d,1
4,231564,99,NW1 7JX,51.537842,-0.142225,W9 1HA,51.526377,-0.188469,4.776,2018-01-01 13:43:45,2018-01-01 16:15:00,2018-01-01 17:00:00,2018-01-01 13:43:46,2018-01-01 15:45:07,20,4.99,0,2,0,0,40.0,30.0,20.0,4.80,1,0,0,99,99,1,99,99,0,1,1,NaN,None,44.0,9.61,1,1,0,NW,NW1,7JX,W,W9,1HA,1,0,0,2018-01-01 14:00:00,6.60,2.65,80,3.60,75,Clouds,04d,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138940,831738,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 12:15:00,2019-07-15 22:15:00,2019-07-14 21:21:37,2019-07-15 10:05:41,40,140.00,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,99,99,99,99,99,NaN,None,77.0,16.00,1,1,0,SE,SE7,7RU,SE,SE1,3PR,1,0,0,2019-07-14 21:00:00,16.41,14.01,67,3.60,1,Clear,01n,0
138941,831739,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 16:15:00,2019-07-15 23:15:00,2019-07-14 21:21:37,2019-07-15 15:15:02,40,134.99,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,99,99,99,99,99,NaN,None,77.0,15.43,1,1,0,SE,SE7,7RU,SE,SE1,3PR,1,0,0,2019-07-14 21:00:00,16.41,14.01,67,3.60,1,Clear,01n,0
138942,831740,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 16:15:00,2019-07-15 23:15:00,2019-07-14 21:21:37,2019-07-15 15:15:02,40,134.99,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,99,99,99,99,99,NaN,None,77.0,15.43,1,1,0,SE,SE7,7RU,SE,SE1,3PR,1,0,0,2019-07-14 21:00:00,16.41,14.01,67,3.60,1,Clear,01n,0
138943,831741,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 16:15:00,2019-07-15 23:15:00,2019-07-14 21:21:37,2019-07-

### Event level
Merge the datasets for analyses on the event level

In [13]:
# additionally merge events 
df_events_merged = data.add_event_outcome(df_jobs_merged, df_events)
# save the dataframe
data.write_feather_file(df_events_merged, utils.path_to('data', 'final', 'df_clean_events.feather'))

Writing feather file to c:\users\thilo\repos\s2ds\data\final\df_clean_events.feather


,job_id,status,pickup_postcode,pickup_location_lat,pickup_location_lng,delivery_postcode,delivery_location_lat,delivery_location_lng,distance,insertion_date,earliest_pickup_time,delivery_deadline,date_booked,date_started,vehicle_type,courier_money_earned_net,is_first_war_job,job_priority,riskiness,show_on_board,size_x,size_y,size_z,weight,special_care,is_food,is_fragile,is_liquid,is_not_rotatable,is_glass,...,is_beef,is_pork,canceled_status,canceled_reason,estimated_journey_time,courier_earnings_calc,event_counts,accepted_count,rejected_count,pickup_district,pickup_postcode_outer,pickup_postcode_inner,delivery_district,delivery_postcode_outer,delivery_postcode_inner,is_accepted,is_rejected,is_hard,dt_iso,temp,feels_like,humidity,wind_speed,clouds_all,weather_main,weather_icon,is_daytime,event,courier_id,insertion_date_history
0,231558,99,SW3 4NX,51.489251,-0.164032,E20 1EJ,51.543535,-0.006004,16.328,2018-01-01 11:22:47,2018-01-01 13:20:00,2018-01-01 16:00:00,2018-01-01 13:16:12,2018-01-01 13:16:16,20,29.42,0,1,0,0,35.0,25.0,2.5,0.10,1,0,1,0,0,0,...,0,0,NaN,None,73.0,19.77,1,1,0,SW,SW3,4NX,E,E20,1EJ,1,0,0,2018-01-01 11:00:00,6.77,4.16,76,1.54,100,Rain,10d,1,accepted,4392,2018-01-01 13:17:46
1,231560,99,NW1 7JX,51.537842,-0.142225,N7 7BY,51.553447,-0.108974,4.196,2018-01-01 12:01:37,2018-01-01 12:15:00,2018-01-01 13:00:00,2018-01-01 12:02:38,2018-01-01 12:02:46,20,9.08,0,0,0,0,40.0,30.0,20.0,9.00,1,99,99,99,99,1,...,1,1,NaN,None,43.0,4.82,1,1,0,NW,NW1,7JX,N,N7,7BY,1,0,0,2018-01-01 12:00:00,7.19,5.38,76,0.51,90,Clouds,04d,1,accepted,10898,2018-01-01 12:03:02
2,231561,99,NW1 7JX,51.537842,-0.142225,W2 5AR,51.515672,-0.196826,5.834,2018-01-01 12:03:34,2018-01-01 15:15:00,2018-01-01 16:00:00,2018-01-01 12:03:35,2018-01-01 14:45:07,15,10.60,0,2,0,0,40.0,30.0,20.0,17.60,1,0,0,99,99,1,...,1,1,NaN,None,65.0,15.47,1,1,0,NW,NW1,7JX,W,W2,5AR,1,0,0,2018-01-01 12:00:00,7.19,5.38,76,0.51,90,Clouds,04d,1,accepted,19425,2018-01-01 14:57:55
3,231562,99,W1T 1BZ,51.518604,-0.132486,W12 7GF,51.508141,-0.219373,8.024,2018-01-01 13:04:13,2018-01-01 13:05:00,2018-01-01 14:45:00,2018-01-01 13:07:01,2018-01-01 13:07:07,20,18.21,0,1,0,0,25.0,16.5,0.5,0.75,0,0,0,0,0,0,...,0,0,NaN,None,53.0,13.13,3,1,2,W,W1T,1BZ,W,W12,7GF,1,0,0,2018-01-01 13:00:00,7.12,2.58,81,4.63,100,Rain,10d,1,rejected,10898,2018-01-01 13:07:25
4,231562,99,W1T 1BZ,51.518604,-0.132486,W12 7GF,51.508141,-0.219373,8.024,2018-01-01 13:04:13,2018-01-01 13:05:00,2018-01-01 14:45:00,2018-01-01 13:07:01,2018-01-01 13:07:07,20,18.21,0,1,0,0,25.0,16.5,0.5,0.75,0,0,0,0,0,0,...,0,0,NaN,None,53.0,13.13,3,1,2,W,W1T,1BZ,W,W12,7GF,1,0,0,2018-01-01 13:00:00,7.12,2.58,81,4.63,100,Rain,10d,1,rejected,4392,2018-01-01 13:08:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325630,831738,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 12:15:00,2019-07-15 22:15:00,2019-07-14 21:21:37,2019-07-15 10:05:41,40,140.00,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,...,99,99,NaN,None,77.0,16.00,1,1,0,SE,SE7,7RU,SE,SE1,3PR,1,0,0,2019-07-14 21:00:00,16.41,14.01,67,3.60,1,Clear,01n,0,accepted,42466,2019-07-15 10:05:48
325631,831739,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 16:15:00,2019-07-15 23:15:00,2019-07-14 21:21:37,2019-07-15 15:15:02,40,134.99,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,...,99,99,NaN,None,77.0,15.43,1,1,0,SE,SE7,7RU,SE,SE1,3PR,1,0,0,2019-07-14 21:00:00,16.41,14.01,67,3.60,1,Clear,01n,0,accepted,18378,2019-07-15 15:51:09
325632,831740,99,SE7 7RU,51.490936,0.030036,SE1 3PR,51.499795,-0.078402,9.856,2019-07-14 21:21:37,2019-07-15 16:15:00,2019-07-15 23:15:00,2019-07-14 21:21:37,2019-07-15 15:15:02,40,134.99,0,0,0,0,23.0,16.5,0.5,0.10,0,99,99,99,99,99,...,99,99,NaN,None,77.0,15.43,1,1,0,SE,SE7,7RU,SE,SE1,3

## Feature extraction

### Job level
Extract features on the job level

Feature extraction is carried out by the module __src/features/generate_features.py__

The function __intermediate_variables__ in the same script file generates variables that have to be transformed again before they can be used as a feature during the feature generation process. 
These variables are:
- hour of the day, day of the week and month of the year for earliest_pickup_time and hour of the day for delivery_deadline
- a recode of the weather_main categories down to 5 categories
- consignment volume, consignment minimum, medium and maximum size, and the sum of minimum and maximum size
- London transport zones derived from the pickup and delivery postcode
- initial time buffer, which is calculated in minutes as delivery_deadline - earliest_pickup_time - estimated_journey_time - 20, 20 being the pickup + delivery time buffer of 10 minutes each

The actual feature generation is carried out by the function __feature_encoding__ thereafter, features are treated dependent on its kind:
- __unchanged features__ simply are passed as is to the feature set
- __individual features__ are defined in the private function ___engineered_variables__. Currently these are three indicators, is_morning_job, is_evening_job and is_scheduled_job
- __log transform features__ have a skewed distribution in the original variable, so the natural logarithm is applied. Variables with minimum values of or below zero are added a constant prior to log which is printed out
- __one hot encoded features__ are categorical features which are recoded in a set of variables according to the number of categories, each set to 1 if the category is met and 0 otherwise. The first call of one hot encoded features are the is_something features for which no extra variable is added for the first category (0), they have category variables for the value 1 and 99 (recode for missing values). The second call yields variables for all categories
- __cyclic features__. These are datetime features which are transformed to a polar representation of an underlying cycle, yielding both a sin and a cos value for the cycle. earliest_pickup_time is transformed to a yearly, weekly and daily cycle, delivery_deadline only to a daily cycle (see http://blog.davidkaleko.com/feature-engineering-cyclical-features.html for details).

In [14]:
# Load job based dataset
df_jobs_merged = pd.read_feather(utils.path_to('data', 'final', 'df_clean_jobs.feather'))

In [15]:
df_job_features, feature_names_job  = features.generate_features(df_jobs_merged)

Timed categorising: earliest_pickup_time
Timed categorising: delivery_deadline
Unchanged features: show_on_board, is_first_war_job, temp, feels_like, humidity, wind_speed, clouds_all, is_daytime
Engineering individual features
Log transforming: distance, initial_time_buffer, estimated_journey_time, courier_earnings_calc, size_min, size_med, size_max, weight, volume, size_min_max
Minimum of initial_time_buffer is less or equal to 0: -12839.0, adding constant of 12840.0 prior to log.
Minimum of courier_earnings_calc is less or equal to 0: -5904.46, adding constant of 5905.46 prior to log.
One hot encoding: is_food, is_fragile, is_liquid, is_not_rotatable, is_glass, is_baked, is_flower, is_alcohol, is_beef, is_pork
One hot encoding: vehicle_type, job_priority, weather_cats, earliest_pickup_time_month, earliest_pickup_time_day, earliest_pickup_time_hour, pickup_zone, delivery_zone
Cyclic encoding: earliest_pickup_time
Cyclic encoding: delivery_deadline


As seen in the output, the variables __initial_time_buffer__ and __courier_earnings_calc__ have some negative values, which shouldn't be the case. For these, further a priori cleaning may be required. After adding a constant as done here, the model may not be affected that much.

### Event level
Extract features on the event level.

Procedure is exactly the same as above as there are no courier features yet.

In [16]:
# Load based dataset
df_events_merged = pd.read_feather(utils.path_to('data', 'final', 'df_clean_events.feather'))

In [17]:
df_events_features, feature_names_event  = features.generate_features(df_events_merged)

Timed categorising: earliest_pickup_time
Timed categorising: delivery_deadline
Unchanged features: show_on_board, is_first_war_job, temp, feels_like, humidity, wind_speed, clouds_all, is_daytime
Engineering individual features
Log transforming: distance, initial_time_buffer, estimated_journey_time, courier_earnings_calc, size_min, size_med, size_max, weight, volume, size_min_max
Minimum of initial_time_buffer is less or equal to 0: -12839.0, adding constant of 12840.0 prior to log.
Minimum of courier_earnings_calc is less or equal to 0: -5904.46, adding constant of 5905.46 prior to log.
One hot encoding: is_food, is_fragile, is_liquid, is_not_rotatable, is_glass, is_baked, is_flower, is_alcohol, is_beef, is_pork
One hot encoding: vehicle_type, job_priority, weather_cats, earliest_pickup_time_month, earliest_pickup_time_day, earliest_pickup_time_hour, pickup_zone, delivery_zone
Cyclic encoding: earliest_pickup_time
Cyclic encoding: delivery_deadline
